**Navies Bayes**

In [2]:
import pandas as pd
import random
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Read the dataset
data = pd.read_csv("D:/explorra/datasets/pima-indians-diabetes.csv",header = None)

In [4]:
# Visualize some data samples from the dataset
data.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# 8th column is the class label
print('\n\n\nStats for the 7 features over the dataset and the 2 classes {8th column}{diabetic/not-diabetic}\n')
print(data.describe())




Stats for the 7 features over the dataset and the 2 classes {8th column}{diabetic/not-diabetic}

                0           1           2           3           4           5  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   62.000000    0.000000    0.000000   27.300000   
50%      3.000000  117.000000   72.000000   23.000000   30.500000   32.000000   
75%      6.000000  140.250000   80.000000   32.000000  127.250000   36.600000   
max     17.000000  199.000000  122.000000   99.000000  846.000000   67.100000   

                6           7           8  
count  768.000000  768.000000  768.000000  
mean     0.471876   33.240885    0.348958  
std      0.331329   11.760232    0.476

In [6]:
TRAIN_TEST_RATIO = 0.8        # 80% training data
picker = list(range(data.shape[0]))        # get all indices as a list
## sometimes the data is arranged classwise and not randomly
## therefore we shuffle the indices
random.shuffle(picker)
trainMax = int(data.shape[0] * TRAIN_TEST_RATIO)

train_features = []
test_features = []
train_labels = []
test_labels = []

for pick in picker[:trainMax]:
    train_features.append(data.values[pick][:-1])
    train_labels.append(int(data.values[pick][-1]))
for pick in picker[trainMax:]:
    test_features.append(data.values[pick][:-1])
    test_labels.append(int(data.values[pick][-1]))

train_features = np.array(train_features)
test_features = np.array(test_features)

In [7]:
data.values[pick]

array([ 9.   , 89.   , 62.   ,  0.   ,  0.   , 22.5  ,  0.142, 33.   ,
        0.   ])

In [41]:
train_features

array([[4.00e+00, 1.20e+02, 6.80e+01, ..., 2.96e+01, 7.09e-01, 3.40e+01],
       [8.00e+00, 1.09e+02, 7.60e+01, ..., 2.79e+01, 6.40e-01, 3.10e+01],
       [0.00e+00, 1.19e+02, 0.00e+00, ..., 3.24e+01, 1.41e-01, 2.40e+01],
       ...,
       [3.00e+00, 1.80e+02, 6.40e+01, ..., 3.40e+01, 2.71e-01, 2.60e+01],
       [5.00e+00, 1.66e+02, 7.60e+01, ..., 4.57e+01, 3.40e-01, 2.70e+01],
       [9.00e+00, 1.64e+02, 8.40e+01, ..., 3.08e+01, 8.31e-01, 3.20e+01]])

In [8]:
print(train_features.shape, len(train_labels), test_features.shape, len(test_labels))

(614, 8) 614 (154, 8) 154


### Exercise 1: Calculate Prior $P(Y)$

The formula for prior has been taught in class. This is also called the class probability. $P(Y)$ or $P(Y = y)$ is the fraction of the elements present in a class

In [9]:
# Get the number of unique classes & corresponding number of elements belonging to each class
classes, counts = np.unique(train_labels, return_counts=True)
print(classes)
print(counts)

[0 1]
[396 218]


In [10]:
### I assume my classes are from 0 ... N for some N (Here, we have just 2 classes)
num_classes = len(classes)
num_feats = train_features.shape[1]  #total number of features
total_samples = len(train_labels)    #total number of samples

### **Exercise 1: Find the prior probability of each class as the list `prior`**

In [11]:
# Prior for any class = {number of samples belonging to that class/ total_samples}
prior = np.array([ x*1.0/total_samples for x in counts ])

In [12]:
print(prior)

[0.64495114 0.35504886]


In [60]:
## Calculate the mean and variance per feature dimension here 
### from the training set from samples belonging to each class label.

means = np.zeros((num_feats, num_classes)) # every feature, for each class
stddev = np.zeros((num_feats, num_classes)) # every feature, for each class

# For each class
for y in classes: # selecting a class 'y'
    pts = train_features[np.where( train_labels == y )[0],:]    # get all samples belonging to 'y'
    # For each feature
    for i in range(num_feats):
        means[i, y] = np.mean(pts[:, i])
        stddev[i, y] = np.std(pts[:, i])

### This completes the training phase
### We know have estimated both the prior probability and the posterior distributions from our training set.

In [120]:
means,stddev

(array([[  3.27777778,   4.83486239],
        [109.87626263, 142.69266055],
        [ 67.72222222,  70.34862385],
        [ 19.93939394,  21.64678899],
        [ 70.1489899 , 101.99541284],
        [ 30.17979798,  35.13807339],
        [  0.42542677,   0.53117431],
        [ 30.88888889,  36.93119266]]),
 array([[  3.00313155,   3.67863597],
        [ 26.5290453 ,  30.91660937],
        [ 18.16308308,  21.86252969],
        [ 14.61963408,  17.75570754],
        [ 98.32737796, 141.45279946],
        [  7.63167548,   7.28467475],
        [  0.29128117,   0.34302411],
        [ 11.40984768,  11.07190176]]))



### Exercise 2: Complete the Gaussian function ###

In [56]:
def gaussian(x, m, v):
    g = np.sqrt(1.0/2*np.pi*v*v)*np.exp( -1.0*(((x - m)/v)**2) )
    return g

### Exercise 3: Find the likelihood for each class 'y', once you have $P(X_{i}|y)$ from Exercise 2 ###

In [57]:
def get_likelihood(point, means, stddev):
    
    feat_prob = np.zeros((num_feats, num_classes))
    for y in classes:
        for i in range(num_feats):
            feat_prob[i, y] = gaussian(point[i], means[i, y], stddev[i, y]) # get the probability
    
    likelihood = np.zeros((num_classes, 1)) # likelihood for each class 'y'
    for y in classes:
        # Take the product of all the feature likelihoods of the class considered
        likelihood[y] = np.prod(feat_prob[np.nonzero(feat_prob), y]) # mutliply for each feature 'Xi'
    return likelihood

## Predict using Naive Bayes classifier

In [58]:
predictions = []
# For each test sample
for i in range(len(test_labels)):
    
    # Get its likelihood of belong to either class
    likelihood = get_likelihood(test_features[i, :], means, stddev)
    
    # Calculate the approximate posterior = likelihood * prior
    approx_posterior = [ np.asscalar(x*y) for x,y in zip(likelihood, prior) ]
    #approx because of missing P(X) (constant) in the denominator
    
    # Make the prediction as that class with the maximum approximate posterior
    prediction = np.argmax(approx_posterior)
    predictions.append(prediction)

In [59]:
print("Accuracy")
print(np.mean([x == y for x, y in zip(predictions, test_labels)]))

Accuracy
0.7077922077922078


### Use same dataset and Implement using a scikit-learn libraries ###

In [70]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


In [71]:
data.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [72]:
X,y=data.iloc[:,:-1],data.iloc[:,-1]

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

In [104]:
X_train.shape,X_test.shape,y_test.shape,y_train.shape

((537, 8), (231, 8), (231,), (537,))

In [105]:
ss=StandardScaler()

In [106]:
Xtrain_std=ss.fit_transform(X_train)

In [107]:
NB=GaussianNB()

In [108]:
NB.fit(Xtrain_std,y_train)

GaussianNB()

In [109]:
Xtest_std=ss.transform(X_test)

In [110]:
y_pred=NB.predict(Xtest_std)

In [111]:
from sklearn.metrics import classification_report

In [112]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       150
           1       0.64      0.62      0.63        81

    accuracy                           0.74       231
   macro avg       0.72      0.72      0.72       231
weighted avg       0.74      0.74      0.74       231



In [86]:
NB.class_prior_

array([0.65175097, 0.34824903])

In [87]:
NB.class_count_

array([335., 179.])

In [53]:
arr=np.random.randint(1,200,(200,8))

In [54]:
print(arr)

[[146 148  13 ... 138 170  32]
 [ 79  45  77 ...  60 175 175]
 [104 134 119 ... 184 152 164]
 ...
 [180  23  50 ...   3  74  68]
 [ 48 102  13 ... 194 199  50]
 [115 144 115 ... 172  15  93]]


In [56]:
arr[1,:]

array([ 79,  45,  77, 135,  82,  60, 175, 175])

In [66]:
abc = train_features[np.where( train_labels == 0 )[0],:]

In [67]:
abc

array([], shape=(0, 8), dtype=float64)